In [90]:
from collections import defaultdict
import numpy as np

data = """
RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE"""

# with open(f"data/2024/12.txt", "r") as f:
#     data = f.read()

garden = np.array([list(d) for d in data.strip().split()])
plants = np.unique(garden)
AROUND = ((-1, 0), (0, -1), (1, 0), (0, 1))
nrows, ncols = garden.shape

garden

array([['R', 'R', 'R', 'R', 'I', 'I', 'C', 'C', 'F', 'F'],
       ['R', 'R', 'R', 'R', 'I', 'I', 'C', 'C', 'C', 'F'],
       ['V', 'V', 'R', 'R', 'R', 'C', 'C', 'F', 'F', 'F'],
       ['V', 'V', 'R', 'C', 'C', 'C', 'J', 'F', 'F', 'F'],
       ['V', 'V', 'V', 'V', 'C', 'J', 'J', 'C', 'F', 'E'],
       ['V', 'V', 'I', 'V', 'C', 'C', 'J', 'J', 'E', 'E'],
       ['V', 'V', 'I', 'I', 'I', 'C', 'J', 'J', 'E', 'E'],
       ['M', 'I', 'I', 'I', 'I', 'I', 'J', 'J', 'E', 'E'],
       ['M', 'I', 'I', 'I', 'S', 'I', 'J', 'E', 'E', 'E'],
       ['M', 'M', 'M', 'I', 'S', 'S', 'J', 'E', 'E', 'E']], dtype='<U1')

In [91]:
plots = defaultdict(list)

def get_plot_coords(garden, loc, plant):
    stack = [loc]
    seen = set()
    
    nrows, ncols = garden.shape
    
    while stack:
        vr, vc = stack.pop()
        seen.add((vr, vc))
        
        next_plants = [
            (vr + ar, vc + ac) for ar, ac in AROUND
            if 0 <= vr + ar < nrows and 0 <= vc + ac < ncols
            and garden[vr + ar, vc + ac] == plant
        ]
        for nr, nc in next_plants:
            if (nr, nc) not in seen:
                stack.append((nr, nc))
    return seen

for plant in plants:
    locs = list(zip(*np.where(garden == plant)))
    
    while locs:
        loc = locs.pop()
        seen = get_plot_coords(garden, loc, plant)
        plots[plant].append(list(seen))
        locs = list(set(locs) - seen)
    
plots

defaultdict(list,
            {'C': [[(4, 4),
               (0, 7),
               (5, 5),
               (3, 4),
               (6, 5),
               (5, 4),
               (1, 8),
               (0, 6),
               (1, 7),
               (3, 3),
               (2, 6),
               (1, 6),
               (2, 5),
               (3, 5)],
              [(4, 7)]],
             'E': [[(8, 8),
               (9, 9),
               (8, 7),
               (6, 9),
               (6, 8),
               (5, 8),
               (4, 9),
               (7, 9),
               (8, 9),
               (9, 8),
               (5, 9),
               (9, 7),
               (7, 8)]],
             'F': [[(3, 8),
               (2, 7),
               (3, 7),
               (0, 9),
               (2, 9),
               (3, 9),
               (4, 8),
               (0, 8),
               (1, 9),
               (2, 8)]],
             'I': [[(7, 4),
               (6, 2),
               (7, 1),
            

In [92]:
perimeters = defaultdict(int)
areas = defaultdict(int)

for p, regions in plots.items():
    for i, r in enumerate(regions):
        areas[f"{p}_{i}"] += len(r)
        for vr, vc in r:
            look = [
                (vr + ar, vc + ac) for ar, ac in AROUND
                if (0 <= vr + ar < nrows and 0 <= vc + ac < ncols and garden[vr + ar, vc + ac] != p)
                or ((vr + ar in (-1, nrows)) or (vc + ac in (-1, ncols)))
            ]
            perimeters[f"{p}_{i}"] += len(look)

areas, perimeters

(defaultdict(int,
             {'C_0': 14,
              'C_1': 1,
              'E_0': 13,
              'F_0': 10,
              'I_0': 14,
              'I_1': 4,
              'J_0': 11,
              'M_0': 5,
              'R_0': 12,
              'S_0': 3,
              'V_0': 13}),
 defaultdict(int,
             {'C_0': 28,
              'C_1': 4,
              'E_0': 18,
              'F_0': 18,
              'I_0': 22,
              'I_1': 8,
              'J_0': 20,
              'M_0': 12,
              'R_0': 18,
              'S_0': 8,
              'V_0': 20}))

In [93]:
price = 0
for p in areas.keys():
    price += areas[p] * perimeters[p]

price

1930

In [94]:
# corner cases
nrows, ncols = garden.shape

# if both coords are not on grid or different plant
outer_corners = [
    ((-1, 0), (0, 1)), 
    ((-1, 0), (0, -1)), 
    ((1, 0), (0, 1)), 
    ((1, 0), (0, -1)), 
]

# if first two are same plant but third is different, cannot be off grid
inner_corners = [
    ((-1, 0), (0, 1), (-1, 1)), 
    ((-1, 0), (0, -1), (-1, -1)), 
    ((1, 0), (0, 1), (1, 1)), 
    ((1, 0), (0, -1), (1, -1)), 
]

sides = defaultdict(int)



for plant, regions in plots.items():
    for i, r in enumerate(regions):
        corners = 0
        for cr, cc in r:
            for (ar0, ac0), (ar1, ac1) in outer_corners:
                or0, oc0 = cr + ar0, cc + ac0
                or1, oc1 = cr + ar1, cc + ac1
                if (
                    ((not 0 <= or0 < nrows or not 0 <= oc0 < ncols) or garden[or0, oc0] != plant) and
                    ((not 0 <= or1 < nrows or not 0 <= oc1 < ncols) or garden[or1, oc1] != plant)
                ):
                    corners += 1
            
            for (ar0, ac0), (ar1, ac1), (ar2, ac2) in inner_corners:
                ir0, ic0 = cr + ar0, cc + ac0
                ir1, ic1 = cr + ar1, cc + ac1
                ir2, ic2 = cr + ar2, cc + ac2
                if (
                    any(not 0 <= xr < nrows for xr in (ir0, ir1, ir2)) or 
                    any(not 0 <= xc < ncols for xc in (ic0, ic1, ic2))
                ):
                    continue
                else:
                    if (
                        garden[ir0, ic0] == plant and 
                        garden[ir1, ic1] == plant and 
                        garden[ir2, ic2] != plant
                    ):
                        corners += 1
        sides[f"{plant}_{i}"] = corners

sides

defaultdict(int,
            {'C_0': 22,
             'C_1': 4,
             'E_0': 8,
             'F_0': 12,
             'I_0': 16,
             'I_1': 4,
             'J_0': 12,
             'M_0': 6,
             'R_0': 10,
             'S_0': 6,
             'V_0': 10})

In [95]:
price = 0
for p in areas.keys():
    price += areas[p] * sides[p]

price

1206